In [205]:
from common import *
import numpy as np
from typing import Callable, Optional, Tuple

def sorted_valid_columns(board: np.ndarray):
    """
    Returns an array of columns that may be played into that are sorted from middle to outside.
    """
    valid_columns = np.where(board[-1,:] == NO_PLAYER)[0]  # check the top row of the board to see where there are open spaces
    mid = int(np.ceil(len(valid_columns)/2))
    sorted_valid = np.empty(len(valid_columns))
    sorted_valid[0::2] = np.flip(valid_columns[:mid])
    sorted_valid[1::2] = valid_columns[mid:]

    return sorted_valid.astype(np.int8)
    
def connected_n(board: np.ndarray, player: BoardPiece):

    return

def heuristic(board: np.ndarray, player: BoardPiece, maximizingPlayer: bool):
    """
    Returns heuristic score of the current gamestate depending on if the player is maximizing or minimizing.
    """
    heur_value = 0
    other_player = PLAYER1 if player == PLAYER2 else PLAYER2

    if check_end_state(board, player) == GameState.IS_WIN:
        heur_value += 1000000000

    if check_end_state(board, other_player) == GameState.IS_WIN:
        heur_value -= 100000000

    if not maximizingPlayer:
        heur_value = -heur_value
    return heur_value

def is_terminal_board(board: np.ndarray, player: BoardPiece):
    """
    Returns True only if the game is over at the current state.
    """
    if check_end_state(board, player) != GameState.STILL_PLAYING:
        return True
    else: 
        return False

def alphabeta(board: np.ndarray, player: BoardPiece, depth: np.int8, maximizingPlayer = True, alpha = np.NINF, beta = np.PINF):
    valid_actions = sorted_valid_columns(board)
    best_action = None
    other_player = PLAYER1 if player == PLAYER2 else PLAYER2

    if (depth == 0) or (is_terminal_board(board, player)):
         return -1, heuristic(board, player, maximizingPlayer)
    if maximizingPlayer:
        value = np.NINF
        for move in valid_actions:
            child = apply_player_action(board, move, player)
            new_value = alphabeta(child, other_player, depth - 1, False, alpha, beta)[1]
            if new_value > value:
                best_action = move
                value = new_value
            if value >= beta:
                break 
            alpha = max(alpha, value)
        return best_action, value
    else:
        value = np.PINF
        for move in valid_actions:
            child = apply_player_action(board, move, player)
            new_value = alphabeta(child, other_player, depth - 1, True, alpha, beta)[1]
            if new_value < value:
                best_action = move
                value = new_value
            if value <= alpha:
                break 
            beta = min(beta, value)
        return best_action, value

def generate_move_minimax(
    board: np.ndarray, player: BoardPiece, saved_state: Optional[SavedState]
) -> Tuple[PlayerAction, Optional[SavedState]]:
    depth = 4
    action, value = alphabeta(board, player, depth, True)
    print('action:', action, 'value: ', value)
    return action, saved_state

In [206]:
from scipy import signal

In [538]:
board1 = initialize_game_state()
board1 = apply_player_action(board1, 3, PLAYER1)
board1 = apply_player_action(board1, 4, PLAYER2)
board1 = apply_player_action(board1, 4, PLAYER1)
board1 = apply_player_action(board1, 5, PLAYER2)
board1 = apply_player_action(board1, 5, PLAYER1)
board1 = apply_player_action(board1, 6, PLAYER2)
board1 = apply_player_action(board1, 5, PLAYER1)
board1 = apply_player_action(board1, 5, PLAYER2)
board1 = apply_player_action(board1, 6, PLAYER1)
board1 = apply_player_action(board1, 0, PLAYER2)
board1 = apply_player_action(board1, 0, PLAYER2)
board1 = apply_player_action(board1, 0, PLAYER2)
board1 = apply_player_action(board1, 0, PLAYER2)


board1 = apply_player_action(board1, 2, PLAYER1)
board1 = apply_player_action(board1, 2, PLAYER1)
board1 = apply_player_action(board1, 2, PLAYER1)



In [539]:
print(pretty_print_board(board1))

|==============|
|              |
|              |
|O         O   |
|O   X     X   |
|O   X   X X X |
|O   X X O O O |
|==============|
|0 1 2 3 4 5 6 |


In [535]:
def connected_n(board: np.ndarray, player: BoardPiece, n: np.int8, num_rows = np.int8(6), num_columns = np.int8(7)):
    count_n = 0
    
    horizontal_kernel = np.ones((1,4),dtype=np.int8)
    vertical_kernel = np.ones((4,1),dtype=np.int8)
    diagonal_kernel = np.eye(4, dtype=np.int8)
    off_diagonal_kernel = np.eye(4, dtype=np.int8)[::-1]
    kernels = [vertical_kernel, horizontal_kernel, off_diagonal_kernel, diagonal_kernel]

    for kernel in kernels:
        kernel_width = kernel.shape[1]
        kernel_height = kernel.shape[0]
        for i in range(num_rows - kernel_height + 1):
            for j in range(num_columns - kernel_width + 1):
                sample_array = board[i:i+kernel_height, j:j+kernel_width][np.where(kernel)]
                if (np.sum(sample_array[sample_array == player]) == player*n) and (len(sample_array[sample_array == NO_PLAYER]) == 4-n):
                    count_n += 1
    return count_n
        

In [540]:
connected_n(board1, PLAYER2, 4)

1

In [541]:
print("\033[0;37;40m Normal text\n")

 Normal text



In [544]:
print("\033[1;31;40m Bright Red" )

 Bright Red
